Import python libraries required for the script

In [1]:
from rfpimp import *
from sklearn.ensemble import RandomForestClassifier #, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import norm

import itertools as it
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
rfc = RandomForestClassifier(n_estimators=300, warm_start=True)


Read the CSV files

In [4]:
df = pd.read_csv('Dataset_BP1_v6_mod01.csv')
print(df.columns)

Index(['RECID', 'CUT_LENGTH', 'CONV_TIME', 'PRESSURE_S1_L', 'PRESSURE_S1_R',
       'PRESSURE_S2_L', 'PRESSURE_S2_R', 'PRESSURE_S3_L', 'PRESSURE_S3_R',
       'PRESSURE_S4_L', 'PRESSURE_S4_R', 'PRESSURE_S5_L', 'PRESSURE_S5_R',
       'COMP_MACHINEID', 'COMP_POSITION_BIN', 'LFT_SPLICE_LAG1',
       'LFT_SPLICE_MA5', 'LFT_SPLICE_MA10', 'LFT_SPLICE_MA20',
       'LFT_SPLICE_SLOPE5', 'LFT_SPLICE_SLOPE10', 'LFT_SPLICE_SLOPE20',
       'MID_SPLICE_LAG1', 'MID_SPLICE_MA5', 'MID_SPLICE_MA10',
       'MID_SPLICE_MA20', 'MID_SPLICE_SLOPE5', 'MID_SPLICE_SLOPE10',
       'MID_SPLICE_SLOPE20', 'RHT_SPLICE_LAG1', 'RHT_SPLICE_MA5',
       'RHT_SPLICE_MA10', 'RHT_SPLICE_MA20', 'RHT_SPLICE_SLOPE5',
       'RHT_SPLICE_SLOPE10', 'RHT_SPLICE_SLOPE20', 'SPLICE_OK'],
      dtype='object')


In [5]:
# pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None) 

df.describe()
# df.astype(bool).sum(axis=0)

,RECID,CUT_LENGTH,CONV_TIME,PRESSURE_S1_L,PRESSURE_S1_R,PRESSURE_S2_L,PRESSURE_S2_R,PRESSURE_S3_L,PRESSURE_S3_R,PRESSURE_S4_L,PRESSURE_S4_R,PRESSURE_S5_L,PRESSURE_S5_R,COMP_POSITION_BIN,LFT_SPLICE_LAG1,LFT_SPLICE_MA5,LFT_SPLICE_MA10,LFT_SPLICE_MA20,LFT_SPLICE_SLOPE5,LFT_SPLICE_SLOPE10,LFT_SPLICE_SLOPE20,MID_SPLICE_LAG1,MID_SPLICE_MA5,MID_SPLICE_MA10,MID_SPLICE_MA20,MID_SPLICE_SLOPE5,MID_SPLICE_SLOPE10,MID_SPLICE_SLOPE20,RHT_SPLICE_LAG1,RHT_SPLICE_MA5,RHT_SPLICE_MA10,RHT_SPLICE_MA20,RHT_SPLICE_SLOPE5,RHT_SPLICE_SLOPE10,RHT_SPLICE_SLOPE20,SPLICE_OK
count,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.00000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000,149615.000000
mean,74819.802633,1707.247612,83.210255,0.078739,0.077514,0.066852,0.053943,0.068973,0.05396,0.068891,0.054188,0.071853,0.052882,2.009598,6.770666,6.770235,6.770005,6.769617,-0.000213,-0.000168,-0.000130,7.193190,7.192638,7.192583,7.192687,-0.000231,-0.000132,-0.000076,6.991023,6.990627,6.990638,6.990634,-0.000139,-0.000058,-0.000069,0.805528
std,43193.483380,2.053829,210.436726,0.063747,0.062463,0.046187,0.043985,0.041516,0.04237,0.042954,0.043032,0.045469,0.043885,0.816331,2.333279,1.745616,1.553421,1.371550,0.547399,0.290131,0.152552,2.018858,1.548320,1.384644,1.240571,0.466106,0.245229,0.132219,2.390093,1.844118,1.650061,1.449438,0.547387,0.295517,0.156441,0.395795
min,1.000000,1609.566000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,-47.200000,-19.923350,-7.883220,-1.809230,-11.169120,-7.705990,-7.705990,-56.381480,-23.493140,-8.926430,-1.373170,-12.375970,-12.263440,-12.263440,-44.709190,-16.385950,-7.765450,-2.709570,-10.962440,-5.512830,-2.855290,0.000000
25%,37418.500000,1706.078000,28.615000,0.050000,0.050000,0.040000,0.010000,0.050000,0.01000,0.040000,0.010000,0.050000,0.010000,1.000000,5.521540,5.769675,5.821155,5.887795,-0.252000,-0.145520,-0.078280,6.143370,6.304440,6.342630,6.392100,-0.207920,-0.114380,-0.062510,5.730070,5.965335,6.007760,6.068885,-0.250240,-0.145350,-0.078490,1.000000
50%,74822.000000,1707.404000,35.617000,0.080000,0.080000,0.070000,0.050000,0.070000,0.05000,0.070000,0.050000,0.070000,0.050000,2.000000,6.829310,6.793860,6.772150,6.746670,0.000670,0.000130,0.000060,7.192110,7.191130,7.184250,7.167850,-0.002110,-0.006340,-0.003980,7.083860,7.061580,7.031480,6.992010,0.005330,0.001790,0.000440,1.000000
75%,112225.500000,1708.582000,70.383000,0.100000,0.100000,0.090000,0.080000,0.100000,0.08000,0.100000,0.080000,0.100000,0.080000,3.000000,8.123630,7.832335,7.743395,7.645050,0.269600,0.145630,0.078360,8.334330,8.131440,8.064070,7.989085,0.191050,0.104510,0.059305,8.429700,8.124225,8.031615,7.915370,0.270640,0.146440,0.083170,1.000000
max,149629.000000,1719.298000,10034.240000,1.300000,2.000000,2.000000,1.000000,0.500000,0.76000,0.800000,0.900000,0.500000,1.000000,3.000000,27.813620,17.524150,15.011400,13.341400,10.691020,5.459880,2.755790,26.034800,20.881120,18.356960,15.392700,13.774690,6.451340,3.048310,27.091790,17.837310,16.429350,15.464310,10.919370,5.272990,2.589630,1.000000


In [9]:
df_temp = df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]
df_temp.describe(include = 'all')

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
df_temp = df[(np.abs(stats.zscore(df)) >= 3).any(axis=1)]
df_temp.describe(include = 'all')

In [ ]:
df_temp = df[df['SPLICE_GRADE']==1]
df_temp.describe(include = 'all')

In [ ]:
column = 'LFT_SPLICE_SLOPE20'
sns.distplot(df_temp[column], fit=norm);
fig = plt.figure()
res = stats.probplot(df_temp[column], plot=plt)

In [ ]:
# df[df['CUT_LENGTH'] < 1650]['SPLICE_GRADE'].value_counts()
 df[df['CONV_WAIT_TIME'] > 1000]['SPLICE_GRADE'].value_counts()


In [ ]:
# df_temp = df
# print(df_temp.shape)
# # Take log of CONV_WAIT_TIME
# df_temp['CONV_WAIT_TIME_LOG'] = np.log(df_temp['CONV_WAIT_TIME'])
# # Remove infinity values after taking log
# df_temp = df_temp.replace([np.inf, -np.inf], np.nan).dropna(subset=["CONV_WAIT_TIME_LOG"], how="all")
# print(df_temp.shape)

# df_temp = df_temp[df_temp['CONV_WAIT_TIME'] < 1000]
df_temp = df[df['CONV_WAIT_TIME'] < 1000]

column = 'CONV_WAIT_TIME_LOG'
sns.distplot(df_temp[column], fit=norm);
fig = plt.figure()
res = stats.probplot(df_temp[column], plot=plt)

In [ ]:
df = df.drop(['SPLICE_OK', 'SPLICE_GRADE', 'MID_SPLICE_GRADE', 'LFT_SPLICE_GRADE', # 'RHT_SPLICE_GRADE', 
              'LFT_SPLICE_LENGTH', 'MID_SPLICE_LENGTH', 'RHT_SPLICE_LENGTH', 
              'LFT_SPLICE_DELTA', 'MID_SPLICE_DELTA', 'RHT_SPLICE_DELTA',              
              'LFT_SPLICE_MA5', 'LFT_SPLICE_MA10', 'LFT_SPLICE_MA20', 'LFT_SPLICE_MA50', 
              'LFT_SPLICE_SLOPE5', 'LFT_SPLICE_SLOPE10', 'LFT_SPLICE_SLOPE20', 'LFT_SPLICE_SLOPE50', 
              'MID_SPLICE_MA5', 'MID_SPLICE_MA10', 'MID_SPLICE_MA20', 'MID_SPLICE_MA50', 
              'MID_SPLICE_SLOPE5', 'MID_SPLICE_SLOPE10', 'MID_SPLICE_SLOPE20', 'MID_SPLICE_SLOPE50',  
              'RHT_SPLICE_MA5', 'RHT_SPLICE_MA10', 'RHT_SPLICE_MA20',  'RHT_SPLICE_MA50', 
              'RHT_SPLICE_SLOPE5', 'RHT_SPLICE_SLOPE10',  'RHT_SPLICE_SLOPE20', 'RHT_SPLICE_SLOPE50',
              'LFT_SPLICE_PREV', 'MID_SPLICE_PREV', 'RHT_SPLICE_PREV',
               'BP1_S8_L', 'BP1_S7_L', 'BP1_S6_L', 'BP1_S6_R', 'BP1_S7_R', 'BP1_S8_R'
             ], axis=1)

print(df.columns)

for col in df.columns:
    if len(df[col].unique()) == 1:
        df = df.drop(col,inplace=True,axis=1)
        
# print(df.columns)
# df['SPLICE_GRADE'].head

Check the columns in the data

In [7]:
print(df.shape)
df = df.drop_duplicates()
print(df.shape)


(149615, 37)
(149615, 37)


In [8]:
print(df.shape)
df = df.drop_duplicates(subset=['CUT_LENGTH', 'CONV_WAIT_TIME', #'BP1_S8_L', 'BP1_S7_L', 'BP1_S6_L', 
                                'BP1_S5_L', 
                                'BP1_S4_L', 'BP1_S3_L', 'BP1_S2_L', 'BP1_S1_ML', 'BP1_S1_MR', 'BP1_S2_R', 
                                'BP1_S3_R', 'BP1_S4_R', 'BP1_S5_R', #'BP1_S6_R', 'BP1_S7_R', 'BP1_S8_R'
                               ])
# df = df.drop_duplicates()
print(df.shape)
# df.head

(149615, 37)


KeyError: Index(['BP1_S1_ML', 'BP1_S1_MR', 'BP1_S2_L', 'BP1_S2_R', 'BP1_S3_L',
       'BP1_S3_R', 'BP1_S4_L', 'BP1_S4_R', 'BP1_S5_L', 'BP1_S5_R',
       'CONV_WAIT_TIME'],
      dtype='object')

In [ ]:
# Variance of the columns - can drop columns based on the variance
df.std() > 0.04
# df.loc[:, df.std() > .02]

In [ ]:
# Count non-zero values
df.astype(bool).sum(axis=0)

In [ ]:
# df[df['CUT_LENGTH'] < 1000]['SPLICE_GRADE'].value_counts
# pd.DataFrame({'count' : df.groupby(['SPLICE_GRADE'])['CUT_LENGTH'].size()}).reset_index()
temp = df.loc[df['CUT_LENGTH'] < 1700]['SPLICE_GRADE']
temp.value_counts()
# temp

In [ ]:
# scaler = MinMaxScaler()
# mycols = ['CUT_LENGTH', 'CONV_WAIT_TIME', #'BP1_S8_L', 'BP1_S7_L', 'BP1_S6_L', 
#           'BP1_S5_L', 'BP1_S4_L', 'BP1_S3_L', 
#           'BP1_S2_L', 'BP1_S1_ML', 'BP1_S1_MR', 'BP1_S2_R', 'BP1_S3_R', 'BP1_S4_R', 'BP1_S5_R', #'BP1_S6_R',
#           #'BP1_S7_R', 'BP1_S8_R', 
#           'PART_LENGTH' #, 'LFT_SPLICE_PREV', 'MID_SPLICE_PREV', 'RHT_SPLICE_PREV'
#          ]
# df[mycols] = scaler.fit_transform(df[mycols])

In [ ]:
# df.std() > 0.04

In [ ]:
# PCA - todo this one and other few

# from sklearn.decomposition import PCA
# X = df.drop(['RHT_SPLICE_GRADE'], axis=1)
# pca = PCA()
# X_r = pca.fit(X).transform(X)

In [ ]:
from imblearn.over_sampling import SMOTE

X = df.drop(['RHT_SPLICE_GRADE'], axis=1)
y = df['RHT_SPLICE_GRADE'].astype('int')

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)
X_train, x2, y_train, y2 = train_test_split(X, y, test_size=0.25, stratify=df['RHT_SPLICE_GRADE'])

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", x2.shape)
print("Number transactions y_test dataset: ", y2.shape)

print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=2)
x1, y1 = sm.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(x1.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y1.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y1==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y1==0)))

print("Test data, counts of label '1': {}".format(sum(y2==1)))
print("Test data, counts of label '0': {}".format(sum(y2==0)))

# x2, y2 = df_test.drop('RHT_SPLICE_GRADE',axis=1), df_test['RHT_SPLICE_GRADE'].astype('int')

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
print(missing_data.head(10))
df = df.dropna()

In [ ]:
# def numericSG(row):
#     if row['SPLICE_GRADE'] == 'Bad':
#         return 0
#     if row['SPLICE_GRADE'] == 'OK' :
#         return 1
#     return 2

# def inverse(x):
#     if x==0: 
#         return 0
#     return 1/x

In [ ]:
# df_train, df_test = train_test_split(df, test_size=0.25, stratify=df['RHT_SPLICE_GRADE'])

# x1, y1 = df_train.drop('RHT_SPLICE_GRADE',axis=1), df_train['RHT_SPLICE_GRADE'].astype('int')
# x2, y2 = df_test.drop('RHT_SPLICE_GRADE',axis=1), df_test['RHT_SPLICE_GRADE'].astype('int')

# print(x1.columns)

In [ ]:
rfc.fit(x1,y1)
y2_RFC_model = rfc.predict(x2)
print("RandomForestClassifier Accuracy :", accuracy_score(y2, y2_RFC_model))
importances2 = rfc.feature_importances_
std2 = np.std([tree.feature_importances_ for tree in rfc.estimators_], axis=0)
indices2 = np.argsort(importances2)[::-1]

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in it.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
target_names = ['Bad', 'Good']
class_names = target_names

In [ ]:

# Compute confusion matrix
cnf_matrix = confusion_matrix(y2, y2_RFC_model)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='RFC Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='RFC Normalized confusion matrix')

plt.show();

In [ ]:
print(" *********** RandomForestClassifier *********** ")
print(classification_report(y2, y2_RFC_model, target_names=target_names, sample_weight=None, digits=4))


In [ ]:
# import numpy as np
# from scipy import interp
# import matplotlib.pyplot as plt

# from sklearn import svm, datasets
# from sklearn.metrics import roc_curve, auc
# from sklearn.model_selection import StratifiedKFold

# # Run classifier with cross-validation and plot ROC curves
# cv = StratifiedKFold(n_splits=6)
# classifier = svm.SVC(probability=True, random_state=7)

# tprs = []
# aucs = []
# mean_fpr = np.linspace(0, 1, 100)

# X = df.drop(['SPLICE_GRADE'], axis=1)
# y = df['SPLICE_GRADE'].astype('int')

# i = 0
# for train, test in cv.split(X, y):
#     probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
#     # Compute ROC curve and area the curve
#     fpr, tpr, thresholds = roc_curve(y[test], probas_[:, 1])
#     tprs.append(interp(mean_fpr, fpr, tpr))
#     tprs[-1][0] = 0.0
#     roc_auc = auc(fpr, tpr)
#     aucs.append(roc_auc)
#     plt.plot(fpr, tpr, lw=1, alpha=0.3,
#              label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

#     i += 1
# plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
#          label='Chance', alpha=.8)

In [ ]:
# from sklearn.externals import joblib
# joblib.dump(rfc, 'rfc_bp1_02_great.pkl', compress=9)

In [ ]:
# from sklearn2pmml import sklearn2pmml
# from sklearn2pmml import make_pmml_pipeline

# pipeline = make_pmml_pipeline(rfc)
# sklearn2pmml(pipeline, "rfc_bp1_rht.pmml", with_repr = True)

In [ ]:
# https://www.marcoaltini.com/blog/dealing-with-imbalanced-data-undersampling-oversampling-and-proper-cross-validation